# What is YOLO?
**YOLO** refers to “You Only Look Once” is one of the most versatile and famous object detection models. For every **real-time object detection** work.

YOLO algorithms divide all the given input images into the SxS grid system. Each grid is responsible for object detection. Now those Grid cells predict the boundary boxes for the detected object. For every box, we have five main attributes: x and y for coordinates, w and h for width and height of the object, and a confidence score for the probability that the box containing the object.
# YOLOv5
YOLOv5 got released Utralytics. It is publicly released on [Github](https://github.com/ultralytics/yolov5). 
Utralytics introduced the YOLOv5 Pytorch based approach, and Yes! YOLOv5 is written in the **Pytorch framework**.That's why it is super fast.
And it comes with some default augmentation technique, which makes help it to converge it toward accuracy

# Comparison
YOLOv5 set the benchmark for object detection models very high. It already surpussed previous state of the art object detection model EfficientDet and other models from YOLO family.

<img src='https://user-images.githubusercontent.com/26833433/114313216-f0a5e100-9af5-11eb-8445-c682b60da2e3.png' height=500 width=800/>

**In the notebook, We'll detect head of the wheat**

<img src='https://i.ytimg.com/vi/yqvMuw-uedU/maxresdefault.jpg' height=500 width=800/>

# Firstly we'll preprocess the dataset in suitable format for YOLOv5.
The format should look like this:

    - converter(main directory)
        - val
            - labels (contains all the box dimensions)
            - images (contains images)
        - train
            - labels
            - images

In [ ]:
# Importing Required packages
import os
import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
import shutil as sh

import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
%matplotlib inline

In [ ]:
# Reading the dataframe
df = pd.read_csv('../input/global-wheat-detection/train.csv')
df.head()

In [ ]:
# read the training data.

df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = (df['x'] + df['w'])/2
df['y_center'] = (df['y'] + df['h'])/2
df['classes'] = 0

# Making new dataframe, suitable to make suitable dataset for yolov5
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df.head()

In [ ]:
index = list(set(df.image_id))
print("Total Images: ",len(index))

In [ ]:
# This cell will automatically make the dataset and save it to convertor folder
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val/'
            else:
                path2save = 'train/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))
    
# Bases on this notebook: https://www.kaggle.com/orkatz2/yolov5-train

# Now it's turn to train the dataset using YOLOv5
Our stategy is to:
1. Clone [yolov5](https://github.com/ultralytics/yolov5.git) directory from github

2. Install the requirements for yolov5

3. Make a [data.yml](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml) file indicating our training directory, validation directory, number of classes, and classname 

4. Chose a model bases on your requirement(YOLOv5s, YOLOv5m, YOLOv5l, YOLOv5x)

5. Change the YOLOv5(..).yml acording to your dataset. As we have only one class, we'll use nc: 1

6. Start the Training

In [ ]:
# Cloning the repo
!git clone https://github.com/ultralytics/yolov5.git
clear_output()

In [ ]:
# Moving the folders to our working directory
!mv ./yolov5/* ./

In [ ]:
# Checking if the files correctly cloned and moved
!ls

In [ ]:
# installing the requirements file
!pip install -r requirements.txt
clear_output()

In [ ]:
#customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# Our image and annotation files are saved into this directory

print(os.listdir("./convertor/fold0"))

In [ ]:
# Making a directory for storing our data.yaml and custom YOLOv5(..).yml files
!mkdir DataFile

In [ ]:
%%writetemplate ./DataFile/data.yaml

train: ./convertor/fold0/images/train # training directory
val: ./convertor/fold0/images/val # validation directory
test: ./convertor/fold0/images/val # I'll use validation directory for test image
nc: 1 # number of class
names: ['Wheat'] # name of the class

Our models structure files are save here. These are used for training [coco dataset](https://cocodataset.org/#home). But for using for our custom dataset, we have to change the nc parameter to 1, insted of 80.

In [ ]:
print(os.listdir("./models"))

In [ ]:
# checking the yolov5s model architecture
!cat ./models/yolov5.yaml

Modifying the yolov5s model architecture for nc: 1

In [ ]:
%%writetemplate ./DataFile/customYOLOv5x.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

With our __data.yaml__ and __custom_yolov5s.yaml__ files ready to go we are ready to train!

To kick off training we running the training command with the following options:

- ```img```: define input image size
- ```batch```: determine batch size
- ```epochs```: define the number of training epochs.
- ```data```: set the path to our yaml file
- ```cfg```: specify our model configuration
- ```weights```: specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)
- ```name```: result names
- ```cache```: cache images for faster training
- ```evolve```: evolve hyperparameters

In [ ]:
# You should skip this line
!wandb off

In [ ]:
%%time
!python train.py --img 640  --batch 16 --epochs 1 --data ./DataFile/data.yaml --cfg ./DataFile/customYOLOv5x.yaml --weights yolov5x.pt  --name Result --cache

# Training is completed.
Results are saved in ./runs/train directory

In [ ]:
# let's check the training result directory.
# Here model and result are saved
!ls -R ./runs/train

In [ ]:
# Analize the training and validation result
Image('./runs/train/Result/results.png')

In [ ]:
# Analize the Confusion matrix
Image('./runs/train/Result/confusion_matrix.png',width=400)

# Let's run detection on test images
Parameters to consider:

- ```img-size```: define input image size
- ```conf```: Minimum threshold of confidence
- ```source```: Location of the image/video file
- ```weights```: specify our trined weight file
- ```augment```: Augmentation of the images during detection for better result

In [ ]:
!python detect.py --img-size 800  --conf 0.2 --source ../input/global-wheat-detection/test --weights ./runs/train/Result/weights/best.pt --augment 

In [ ]:
# Detected images/video are saved in this path
!ls -R runs/detect

In [ ]:
Image('runs/detect/exp/2fd875eaa.jpg',width=400)

# Techniques to push the training result

* Experimenting with hyper parameters
* increasing the image size

In [ ]:
# hyper parameters data are saved in this location
!ls ./data

**Let's train the model with better hyper parameter**

In [ ]:
%cat ./data/hyp.finetune.yaml

In [ ]:
%%writetemplate ./DataFile/hyp.custom.yaml

lr0: 0.0032
lrf: 0.12
momentum: 0.843
weight_decay: 0.00036
warmup_epochs: 2.0
warmup_momentum: 0.5
warmup_bias_lr: 0.05
box: 0.0296
cls: 0.243
cls_pw: 0.631
obj: 0.301
obj_pw: 0.911
iou_t: 0.2
anchor_t: 2.91
# anchors: 3.63
fl_gamma: 0.0   # 
hsv_h: 0.0138
hsv_s: 0.002
hsv_v: 0.2
degrees: 0.2
translate: 0.0
scale: 0.3
shear: 0.0
perspective: 0.0
flipud: 0.00856
fliplr: 0.5
mosaic: 1.0
mixup: 0.5

- ```--hyp```: Location of new custom hyper parameter

# I've used weight of previous trained model.So, this training will initilize from the best result(weight) of the previous model

In [ ]:
%%time
!python train.py --img 1024  --batch 4 --epochs 1 --data ./DataFile/data.yaml --cfg ./DataFile/customYOLOv5x.yaml --hyp ./DataFile/hyp.custom.yaml  --weights ./runs/train/Result/weights/best.pt  --name FineTuned

In [ ]:
# new training result
os.listdir('./runs/train')

In [ ]:
!ls -R ./runs/train/FineTuned

# Model Ensembling
I've used two models during detection. It'll increase detection result.

In [ ]:
!python detect.py  --img 800 --weights ./runs/train/Result/weights/best.pt ./runs/train/FineTuned/weights/best.pt --source ../input/global-wheat-detection/test

In [ ]:
!ls runs/detect

In [ ]:
!ls -R runs/detect/exp

In [ ]:
Image('runs/detect/exp/2fd875eaa.jpg', width=400)

# TTA(Test Time Augmentation) and Model Ensembling

In [ ]:
!python test.py --weights ./runs/train/Result/weights/best.pt ./runs/train/FineTuned/weights/best.pt --data ./DataFile/data.yaml --img 800 --augment

In [ ]:
!ls -R runs/test/exp

In [ ]:
# That's all for today, I'll update the notebook for pseudo labelling soon.
# Please let me know if you face any issue or want have any confusion